In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape) 


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [22]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  #加入了num of channels.
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

#在初始化时也要使channel相对应，同时出现新的参数depth相当于输出层的类数。
#depth的具体作用和如何运算的还有待考证。是不是代表在经过conv的运算后得到了
#新的矩阵，再对新的矩阵进行运算得到第一层的结果。  感觉又有可能是用16个不同的矩阵与之做conv操作。
#特征共享代表卷积经过的某一个区域，他们使用相同的权值进行后面的计算。比如28*28 的图与5*5的图进行卷积步长s为2，那么图像的大小为14*14。
#新生成的14*14的图中每个元素中含有原图中5*5=25个像素的信息，那么这25个像素就共享这些权值。我觉得层数逇意思就是使用多个5*5的kenel进行卷积。
#以来保存更多的信息。
#The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32].
#Second Convolutional Layer In order to build a deep network, we stack several layers of this type.
#The second layer will have 64 features for each 5x5 patch.

  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))   
  layer1_biases = tf.Variable(tf.zeros([depth]))      
  layer2_weights = tf.Variable(tf.truncated_normal(               
      [patch_size, patch_size, depth, depth], stddev=0.1))     #第二层的channel数量与第一层的depth相同。
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #预设的stride步数为2加上same padding因此得到输出大小缩小4倍。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') # [1, 2, 2, 1] 1 1是固定的。2 2 分别代表横向和纵向的步长。
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.827135
Minibatch accuracy: 9.4%
Validation accuracy: 12.9%
Minibatch loss at step 1000: 0.471241
Minibatch accuracy: 83.6%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 0.364410
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 0.407513
Minibatch accuracy: 82.0%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.449184
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 5000: 0.502810
Minibatch accuracy: 81.2%
Validation accuracy: 89.0%
Minibatch loss at step 6000: 0.231359
Minibatch accuracy: 93.0%
Validation accuracy: 89.3%
Minibatch loss at step 7000: 0.343631
Minibatch accuracy: 90.6%
Validation accuracy: 89.5%
Minibatch loss at step 8000: 0.425353
Minibatch accuracy: 90.6%
Validation accuracy: 89.8%
Minibatch loss at step 9000: 0.344846
Minibatch accuracy: 90.6%
Validation accuracy: 89.7%
Minibatch loss at step 10000: 0.435164
Minibatch accuracy: 84.4%
Validation accura

In [26]:
batch_size = 128
patch_size = 5
depth = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  #加入了num of channels.
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

#在初始化时也要使channel相对应，同时出现新的参数depth相当于输出层的类数。
#depth的具体作用和如何运算的还有待考证。是不是代表在经过conv的运算后得到了
#新的矩阵，再对新的矩阵进行运算得到第一层的结果。  感觉又有可能是用16个不同的矩阵与之做conv操作。
#特征共享代表卷积经过的某一个区域，他们使用相同的权值进行后面的计算。比如28*28 的图与5*5的图进行卷积步长s为2，那么图像的大小为14*14。
#新生成的14*14的图中每个元素中含有原图中5*5=25个像素的信息，那么这25个像素就共享这些权值。我觉得层数逇意思就是使用多个5*5的kenel进行卷积。
#以来保存更多的信息。
#The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32].
#Second Convolutional Layer In order to build a deep network, we stack several layers of this type.
#The second layer will have 64 features for each 5x5 patch.

  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))   
  layer1_biases = tf.Variable(tf.zeros([depth]))      
  layer2_weights = tf.Variable(tf.truncated_normal(               
      [patch_size, patch_size, depth, depth], stddev=0.1))     #第二层的channel数量与第一层的depth相同。
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #预设的stride步数为2加上same padding因此得到输出大小缩小4倍。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.

  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') # [1, 2, 2, 1] 1 1是固定的。2 2 分别代表横向和纵向的步长。
    hidden_1 = tf.nn.relu(conv + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool_2.get_shape().as_list()
    
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden1_drop = tf.nn.dropout(hidden1, 0.5)
    
    hidden2 = tf.nn.relu(tf.matmul(hidden1_drop, layer4_weights) + layer4_biases)
    hidden2_drop = tf.nn.dropout(hidden2, 0.9)
    
    return tf.matmul(hidden2, layer5_weights) + layer5_biases

    
    
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.678959
Minibatch accuracy: 8.6%
Validation accuracy: 10.3%
Minibatch loss at step 1000: 0.622687
Minibatch accuracy: 79.7%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 0.518190
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.543146
Minibatch accuracy: 81.2%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 0.596956
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 0.604891
Minibatch accuracy: 77.3%
Validation accuracy: 86.5%
Minibatch loss at step 6000: 0.279377
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 7000: 0.504184
Minibatch accuracy: 83.6%
Validation accuracy: 87.0%
Minibatch loss at step 8000: 0.632698
Minibatch accuracy: 83.6%
Validation accuracy: 87.2%
Minibatch loss at step 9000: 0.369810
Minibatch accuracy: 86.7%
Validation accuracy: 87.7%
Minibatch loss at step 10000: 0.436824
Minibatch accuracy: 87.5%
Validation accura

In [33]:
batch_size = 128
patch_size = 5
depth1 = 64
depth2 = 128
num_hidden1 = 512
num_hidden2 = 100


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  #加入了num of channels.
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

#在初始化时也要使channel相对应，同时出现新的参数depth相当于输出层的类数。
#depth的具体作用和如何运算的还有待考证。是不是代表在经过conv的运算后得到了
#新的矩阵，再对新的矩阵进行运算得到第一层的结果。  感觉又有可能是用16个不同的矩阵与之做conv操作。
#特征共享代表卷积经过的某一个区域，他们使用相同的权值进行后面的计算。比如28*28 的图与5*5的图进行卷积步长s为2，那么图像的大小为14*14。
#新生成的14*14的图中每个元素中含有原图中5*5=25个像素的信息，那么这25个像素就共享这些权值。我觉得层数逇意思就是使用多个5*5的kenel进行卷积。
#以来保存更多的信息。
#The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32].
#Second Convolutional Layer In order to build a deep network, we stack several layers of this type.
#The second layer will have 64 features for each 5x5 patch.

  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))   
  layer1_biases = tf.Variable(tf.zeros([depth1]))  

  layer2_weights = tf.Variable(tf.truncated_normal(               
      [patch_size, patch_size, depth1, depth2], stddev=0.1))     #第二层的channel数量与第一层的depth相同。
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth2, num_hidden1], stddev=0.1)) #预设的stride步数为2加上same padding因此得到输出大小缩小4倍。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))

  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.

  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') # [1, 2, 2, 1] 1 1是固定的。2 2 分别代表横向和纵向的步长。
    hidden_1 = tf.nn.relu(conv + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden1_drop = tf.nn.dropout(hidden1, 0.6)
    
    hidden2 = tf.nn.relu(tf.matmul(hidden1_drop, layer4_weights) + layer4_biases)
    hidden2_drop = tf.nn.dropout(hidden2, 1)
    
    return tf.matmul(hidden2, layer5_weights) + layer5_biases

    
    
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  start_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 10000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.891819
Minibatch accuracy: 9.4%
Validation accuracy: 10.3%
Test accuracy: 10.4%
Minibatch loss at step 1000: 0.468888
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Test accuracy: 90.6%
Minibatch loss at step 2000: 0.373040
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Test accuracy: 92.2%
Minibatch loss at step 3000: 0.443152
Minibatch accuracy: 88.3%
Validation accuracy: 87.4%
Test accuracy: 92.9%
Minibatch loss at step 4000: 0.550190
Minibatch accuracy: 85.2%
Validation accuracy: 87.8%
Test accuracy: 93.5%
Minibatch loss at step 5000: 0.510030
Minibatch accuracy: 83.6%
Validation accuracy: 88.6%
Test accuracy: 93.9%
Minibatch loss at step 6000: 0.233316
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Test accuracy: 94.0%
Minibatch loss at step 7000: 0.395236
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Test accuracy: 94.5%
Minibatch loss at step 8000: 0.419125
Minibatch accuracy: 88.3%
Validation accuracy: 89.5%
Test

In [13]:
batch_size = 128
patch_size = 5
depth1 = 32
depth2 = 32
depth3 = 64
depth4 = 64
num_hidden1 = 1024
num_hidden2 = 512
num_hidden3 = 256



graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  #加入了num of channels.
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

#在初始化时也要使channel相对应，同时出现新的参数depth相当于输出层的类数。
#depth的具体作用和如何运算的还有待考证。是不是代表在经过conv的运算后得到了
#新的矩阵，再对新的矩阵进行运算得到第一层的结果。  感觉又有可能是用16个不同的矩阵与之做conv操作。
#特征共享代表卷积经过的某一个区域，他们使用相同的权值进行后面的计算。比如28*28 的图与5*5的图进行卷积步长s为2，那么图像的大小为14*14。
#新生成的14*14的图中每个元素中含有原图中5*5=25个像素的信息，那么这25个像素就共享这些权值。我觉得层数逇意思就是使用多个5*5的kenel进行卷积。
#以来保存更多的信息。
#The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32].
#Second Convolutional Layer In order to build a deep network, we stack several layers of this type.
#The second layer will have 64 features for each 5x5 patch.

  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))   
  layer1_biases = tf.Variable(tf.zeros([depth1]))  

  layer2_weights = tf.Variable(tf.truncated_normal(               
      [patch_size, patch_size, depth1, depth3], stddev=0.1))     #第二层的channel数量与第一层的depth相同。
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth3]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth3, num_hidden1], stddev=np.sqrt(2.0/12544))) #预设的stride步数为2加上same padding因此得到输出大小缩小4倍。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=np.sqrt(2.0/1024)))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))

  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_hidden3], stddev=np.sqrt(2.0/512)))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden3]))

  layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden3, num_labels], stddev=np.sqrt(2.0/256)))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.

  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') # [1, 2, 2, 1] 1 1是固定的。2 2 分别代表横向和纵向的步长。
    hidden_1 = tf.nn.relu(conv + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')  # 5*5  16
    
    
    shape = pool_2.get_shape().as_list()
    
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden1_drop = tf.nn.dropout(hidden1, 0.8)
    
    hidden2 = tf.nn.relu(tf.matmul(hidden1_drop, layer4_weights) + layer4_biases)
    hidden2_drop = tf.nn.dropout(hidden2, 0.9)
    
    hidden3 = tf.nn.relu(tf.matmul(hidden2_drop, layer5_weights) + layer5_biases)
    hidden3_drop = tf.nn.dropout(hidden3, 1)
    
    return tf.matmul(hidden3_drop, layer6_weights) + layer6_biases

    
    
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  start_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 10000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.983027
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 1000: 0.364473
Minibatch accuracy: 90.6%
Validation accuracy: 86.5%
Test accuracy: 92.1%
Minibatch loss at step 2000: 0.267885
Minibatch accuracy: 93.0%
Validation accuracy: 88.7%
Test accuracy: 93.7%
Minibatch loss at step 3000: 0.325896
Minibatch accuracy: 91.4%
Validation accuracy: 89.6%
Test accuracy: 94.6%
Minibatch loss at step 4000: 0.429482
Minibatch accuracy: 88.3%
Validation accuracy: 89.7%
Test accuracy: 94.7%
Minibatch loss at step 5000: 0.398160
Minibatch accuracy: 85.9%
Validation accuracy: 90.6%
Test accuracy: 95.2%
Minibatch loss at step 6000: 0.177025
Minibatch accuracy: 93.8%
Validation accuracy: 91.0%
Test accuracy: 95.5%
Minibatch loss at step 7000: 0.250572
Minibatch accuracy: 91.4%
Validation accuracy: 91.0%
Test accuracy: 95.8%
Minibatch loss at step 8000: 0.312163
Minibatch accuracy: 89.1%
Validation accuracy: 91.0%
Test 